Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [131]:
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from functools import reduce
from copy import copy
from scipy import sparse
import math

In [132]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [133]:
PROBLEM_SIZE=100
NUM_SETS=100
x = make_set_covering_problem(PROBLEM_SIZE, NUM_SETS, .3)

In [160]:
def fitness1(state,x):
    cost = sum(state)
    valid = np.all(
        reduce(
            np.logical_or,
            [x.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

def fitness2(state,x):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [x.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

fitness = fitness2

def tweak(state):
    new_state = copy(state)
    index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]

#Tabu search
DIM_TABU_LIST=100
neighbours=[]
tabu_list=[DIM_TABU_LIST]
step_done=0
iter_for_best=0
best_state_value=fitness2(current_state,x)
best_state = current_state
for iterations in range(200):
    neighbours = [tweak(current_state) for _ in range(100)]
    max_neig = neighbours[0]
    for neig in neighbours:
        if neig not in tabu_list and fitness2(neig,x) >= fitness2(max_neig,x):
            max_neig=neig
    current_state=max_neig
    step_done +=1
    tabu_list.append(current_state)
    if fitness2(current_state,x) > best_state_value:
        best_state = current_state
        iter_for_best=step_done
        best_state_value = fitness2(current_state,x)
    if len(tabu_list) == DIM_TABU_LIST:
        tabu_list.pop()
print(step_done)
print(iter_for_best)
print(best_state,fitness2(best_state,x))  #final state



200
6
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False] (100, -6)


In [165]:
#iterated local search
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
steady_state_cont = 0
steady_state_limit = 10
step_done=0
iter_for_best=0
for iterations in range(200):
    step_done+=1
    new_state = tweak(current_state)
    if fitness2(new_state,x) >= fitness2(current_state,x):
        steady_state_cont = 0
        current_state = new_state
        #print(current_state, fitness2(current_state,x))
    else:
        steady_state_cont+=1
        if steady_state_cont == steady_state_limit :
            iter_for_best=step_done
            temp_state = [choice([True, False]) for _ in range(PROBLEM_SIZE)]
            if fitness2(temp_state,x) > fitness2(current_state,x):
                current_state=temp_state
            steady_state_cont=0
print(step_done)
print(iter_for_best)
print(current_state,fitness2(current_state,x))

200
191
[False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False] (100, -8)


In [166]:
#simulated annealing
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
cooling_rate=0.97
cost=fitness2(current_state,x)
best_solution=current_state
best_cost=cost
temperature=cooling_rate
last_valid_solution=None
step_done=0
iter_for_best=0
while temperature > 0.01:
    new_state = tweak(current_state)
    step_done+=1
    if  fitness2(new_state,x) >= fitness2(current_state,x) or random() < math.exp( -(fitness2(current_state,x)[1]-fitness2(new_state,x)[1])/ temperature):
        current_state = new_state
        if fitness2(new_state,x)>fitness2(best_solution,x):
            best_solution = current_state
            best_cost = fitness2(new_state,x)
            iter_for_best=step_done
        if fitness2(current_state,x)[0] == PROBLEM_SIZE: #verifico se la soluzione è valida
            last_valid_solution = current_state
    temperature *= cooling_rate
if fitness2(best_solution,x) != PROBLEM_SIZE : 
    best_solution=last_valid_solution
print(step_done)
print(iter_for_best)
print(fitness2(best_solution,x))
print(fitness2(last_valid_solution,x))
print(best_solution)

151
38
(100, -12)
(100, -12)
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False]
